In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

# plt.style.use('seaborn-colorblind')
# %matplotlib inline

# Масштабирование признаков

**Определение:** *Масштабирование признаков* - это метод, используемый для стандартизации диапазона независимых переменных или признаков данных. В обработке данных это также известно как нормализация данных и обычно выполняется на этапе предварительной обработки данных.

### Зачем важно масштабирование признаков

Если диапазон входных данных изменяется, то в некоторых алгоритмах функции потерь не будут работать должным образом.

- Градиентный спуск сходится намного быстрее при выполнении масштабирования признаков. Градиентный спуск - это распространенный алгоритм оптимизации, используемый в логистической регрессии, методе опорных векторов (SVM), нейронных сетях и т. д.
- Алгоритмы, которые включают вычисление расстояний, такие как k-ближайших соседей (KNN), кластеризация, также зависят от масштаба признаков. Рассмотрите, как вычисляется евклидово расстояние: берется корень квадратный из суммы квадратов различий между наблюдениями. Это расстояние может сильно зависеть от различий в масштабе между переменными. Переменные с большими дисперсиями оказывают большее воздействие на это измерение, чем переменные с маленькими дисперсиями.


## Загрузка данных

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('./data/titanic.csv', usecols=use_cols)


In [3]:
data.head(3)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data[["Pclass", "Sex", "SibSp", "Fare"]], data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 4), (268, 4))

## Нормализация/Стадартизация/Z стандартизация

Убираем среднее и мастабируем на стандартное отклонение

<br />z = (X - X.mean) /  std

In [5]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_zscore'] = ss.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Pclass     Sex  SibSp     Fare  Fare_zscore
857       1    male      0  26.5500    -0.122530
52        1  female      1  76.7292     0.918124
386       3    male      5  46.9000     0.299503
124       1    male      0  77.2875     0.929702
578       3  female      1  14.4583    -0.373297
549       2    male      1  36.7500     0.089005


In [6]:
# проверка что mean=0 и std=1
print(X_train_copy['Fare_zscore'].mean())
print(X_train_copy['Fare_zscore'].std())

5.916437306188636e-17
1.0008035356861


## Min-Max scaling
Мастштабировиение по min/max


К [0,1]  <br />X_scaled = (X - X.min / (X.max - X.min)

In [7]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_minmax'] = mms.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Pclass     Sex  SibSp     Fare  Fare_minmax
857       1    male      0  26.5500     0.051822
52        1  female      1  76.7292     0.149765
386       3    male      5  46.9000     0.091543
124       1    male      0  77.2875     0.150855
578       3  female      1  14.4583     0.028221
549       2    male      1  36.7500     0.071731


In [8]:
# Проверим что min = 0, max = 1
print(X_train_copy['Fare_minmax'].max())
print(X_train_copy['Fare_minmax'].min())

1.0
0.0


## Robust scaling
Убираем медиану и мастабирование с Interquartile Ranges Rule

<br />X_scaled = (X - X.median) / IQR

In [9]:
# add the new created feature
from sklearn.preprocessing import RobustScaler
rs = RobustScaler().fit(X_train[['Fare']])
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_robust'] = rs.transform(X_train_copy[['Fare']])
print(X_train_copy.head(6))

     Pclass     Sex  SibSp     Fare  Fare_robust
857       1    male      0  26.5500     0.492275
52        1  female      1  76.7292     2.630973
386       3    male      5  46.9000     1.359616
124       1    male      0  77.2875     2.654768
578       3  female      1  14.4583    -0.023088
549       2    male      1  36.7500     0.927011


In [10]:
# check the range of Fare_minmax
print(X_train_copy['Fare_robust'].max())
print(X_train_copy['Fare_robust'].min())

21.196769312733085
-0.6393180607352158


Если ваш признак не имеет нормальное распределение, например, имеет скошенное распределение или содержит выбросы, то нормализация и стандартизация не являются хорошими выборами, так как они сжимают большинство данных в узкий диапазон.

- Тем не менее, мы можем преобразовать признак в нормально распределенный и затем использовать нормализацию и стандартизацию. 
- При вычислении расстояний или ковариации (алгоритмы, такие как кластеризация, PCA и LDA), лучше использовать нормализацию и стандартизацию, так как это устраняет влияние масштабов на дисперсию и ковариацию. Объяснение можно найти [здесь](ссылка).

Масштабирование Min-Max имеет те же недостатки, что и нормализация и стандартизация, и новые данные могут не ограничиваться в диапазоне [0,1], так как они могут выходить за исходный диапазон. 